___
# EXERCICE FINAL
___

## 1 – Introduction

Cet exercice a pour objectif de vous amener à traiter des données issues d’une expérience en Sciences Cognitives à l’aide d’un script Python. 

Cette expérience cherche à voir s’il existe un biais lié au genre dans notre perception du corps humain. Pour résumer, nous allons chercher à voir si les femmes ont tendance à juger les corps féminins plus gros que les corps masculins, et nous faisons également l’hypothèse que cette tendance serait uniquement féminine. 

L’idée qui sous-tend ces hypothèses serait que les femmes sont exposées, à travers notamment les medias, à des « role models » très minces et à ce qui est coutume d’appeler la « dictature de la minceur ». Cette pression serait plus forte sur les femmes que sur les hommes. En clair, nous allons voir s’il existe un biais de perception lié au genre qui amènerait plus les femmes à percevoir Beyoncé « grosse » …

## 2 - Psychophysique très rapide

La question de la relation entre stimulus externe et perception est éminemment subjective, et de nature probabiliste. 

Ainsi, lorsqu’il s’agit de répondre à la question « avez-vous perçu un flash ? », à intensité lumineuse égale, deux personnes peuvent ne pas répondre de la même manière : l’une peut détecter le flash, l’autre non, et cette différence de réponse ne sera pas la même selon les intensités du flash. Idem, une même personne peut, pour deux flashes identiques, percevoir dans un cas et pas dans un autre. Toute cette relation entre stimulus et perception est le domaine de la psychophysique. 

Si nous étions des « détecteurs parfaits », nous aurions une réponse perceptive constante dès lors que l’intensité d’un stimulus dépasse ce que l’on appelle le « seuil perceptif ». 

![alt text](img/PES_figure.bmp)

Le Point d’Equivalence Subjectif est le niveau d’intensité du stimulus qui déclenche 50% de détection de la part du sujet.

![alt text](img/PES_figure2.bmp)

Cet exercice a pour but de déterminer les PES chez les participants, homme et femme, lorsqu’ils jugent des silhouettes masculines et féminines, et de les comparer.

On attend les résultats suivants (hypothèses non argumentées :-)

+ Les PES des femmes sont plus bas que ceux des hommes (effet du facteur « genre du participant »)
+ Les silhouettes féminines engendrent un PES plus bas que les silhouettes masculines (effet du facteur « type de silhouette »)
+ L’écart silhouettes féminines / masculines est plus important chez les femmes que chez les hommes (interaction entre facteur « genre du participant » et « type de silhouette »)


# Traitement des données

L'analyse va se dérouler en 5 étapes.

Il est évident que l'on va mettre en pratique, la totalité des outils de la semaine.

Voici un guide pour les étapes.

Les datas se trouve dans le zip **Data_exo_final.zip** sur github.

# Etape 1: moissonage des données

Conseils:
  * Inspecter bien le zip et ses fichiers.
  * Construire un grand DataFrame avec la totalité des données brutes. Nom du tableau = **data**

Contraintes:
  * On enlève tous le réponses dont le TR (temps de réponses) est supérieurs à 3s.

Guide pour les noms de variables et colonnes:
  * On garde les noms existant dans ls csv
  * Il va falloir ajouter/calcluer les colonnes suivantes:
    * Subject
    * SubjectGender
    * ItemGender
    * ItemSize

Pour les bons:
  * reformater les nom des sujets avec 2 chiffres.

# Etape 2: calcul des réponses par sujet pour chaque silhouette

Conseils:
  * Construire un DataFrame intermédiaire qui contabilise le nombre de gros/mince pour chaque sujet et silouette.
    Puis la proportion nb_gros/nb_total pour chaque.
    Nom du tableau = **evaluations**






# Etape3 : Graphe des données individuelles

Balladez vous chez seaborn gallery et affichez quelque chose de malins avec le DataFrame **evaluations**

Par example, pour chaque sujet et silhouette un truc de ce style:
![alt text](img/PES_figure3.bmp)

Contraintes:
  * en 4 lignes SVP.


# Etape 4: Fit de la courbe psychometrique (avec interpolation) et estimation du PES

Dans les grands lignes, il faudrait approximer ces données réelles par une courbe psychométrique qui décrit au mieux les observations réelles. Théoriquement il faudrait utilser un Modèle Linéaire Général (GLM) avec un regression binomial.
A méditer: https://fr.wikipedia.org/wiki/R%C3%A9gression_logistique

Mais pour gagner du temps, on va utilser une interpolation pour fitter la courbe psychometrique.

Le PES est le « x_model » qui correspond à un « y_model » de 0.5. (50% de réponses « mince »). Nous allons donc tâcher de déterminer ce x_model.
Dans un premier temps, essayez d’en faire une lecture sur le graphe p.
Ensuite, nous allons déterminer le x_model qui amène la réponse la plus proche possible de 0.5. Ce ne sera pas 0.5 précisément, mais, pour ne pas trop compliquer les choses, nous nous contenterons de cette approximation.

A faire:
  * utilisez la fonction argmin pour chercher le point y_model le plus proche de 0.5 : Y_model_proche_de_0.5
  * pour ca, vous aurez besoin de la valeur absolue de la différence à 0.5
  * le x_model qui correspond à cet y_model_proche_de_0.5 : C’est le PES !
    (vérifiez qu’il correspond bien à l’estimation graphique que vous en avez faite)

Contrainte:
  * Mettre toutes les **pes** dans un DataFrame **all_pes **
  * Bien organiser ce tableau.
 

# Etape 5 : analyse de groupe

A partir du tableau **all_pes**:

  * Faire la moyenne des PES en fonction du genre des participants. Représenter graphiquement ce résultat.
  * Faire la moyenne des PES en fonction du type de silhouette. Représenter graphiquement ce résultat.
  * Faire la moyenne des PES en fonction du type de silhouette et du genre du participant. Représenter graphiquement ce résultat.
  * Faire un model a partir du dernier  résultats.




# Etape 1: lire les fichiers

  1. On extrait le zip la où il est. Soit à la main. Soit mieux sous python.
  2. On parcours l'arborescence avec glob
  3. On lit avec pandas.read_csv
  4. On empile dans une liste
  5. On concat le tout.

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
import zipfile
zipname = '../data/Data_exo_final.zip'
f = zipfile.ZipFile(zipname)
f.extractall('.')

In [13]:
import glob
import re

data_path = './Data_exo_final/'
all_data = []

pattern1 = data_path + '*Sujet*[FH]'
for dirname in glob.glob(pattern1):
    #print(dirname)
    #boucle sur les répertoires des sujet
    subject, subject_gender = re.findall('Sujet (\d+)\-([F|H])', dirname)[0]
    subject = '{:02d}{}'.format(int(subject), subject_gender)
    
    filenames_per_subject = glob.glob(dirname+'/*.csv')
    for filename in filenames_per_subject:
        #print(filename)
        df = pd.read_csv(filename, sep=';', header=0)
        df['Subject'] = subject
        df['SubjectGender'] = subject_gender
        all_data.append(df)
data = pd.concat(all_data)
data

,Numero,Item,Evaluation,RT,Subject,SubjectGender
0,1,M6,gros,1.548,05F,F
1,2,M1,mince,1.375,05F,F
2,3,M5,gros,1.282,05F,F
3,4,W4,gros,1.563,05F,F
4,5,M4,mince,1.454,05F,F
5,6,W7,gros,1.393,05F,F
6,7,W1,mince,1.282,05F,F
7,8,M2,mince,1.607,05F,F
8,9,M3,mince,1.344,05F,F
9,10,W6,gros,1.267,05F,F


In [15]:
# On ne garde que les RT<3.0s

# ce qe l'on perd
print('on enleve', data[data['RT']>=3.].shape[0])

# ce qu'on garde
data = data[data['RT']<3.]



on enleve 0


In [16]:
#Sous excel pour la posetrité
data.to_excel('data_subject.xlsx')

# Etape 2: calcul des reponses par Sujet/Item

In [20]:
evaluations = data.groupby(['Subject', 'Item', 'Evaluation'])['Subject'].count()
evaluations = evaluations.unstack('Evaluation')
evaluations


Evaluation    gros  mince
Subject Item             
01F     M1     NaN    8.0
        M2     NaN    9.0
        M3     NaN    9.0
        M4     1.0    7.0
        M5    10.0    NaN
        M6    10.0    NaN
        M7    10.0    NaN
        M8     9.0    NaN
        M9    10.0    NaN
        W1     NaN    7.0
        W2     NaN   10.0
        W3     NaN   10.0
        W4     1.0    8.0
        W5     8.0    NaN
        W6    10.0    NaN
        W7     8.0    NaN
        W8    10.0    NaN
        W9    10.0    NaN
02F     M1     NaN   10.0
        M2     NaN    9.0
        M3     NaN   10.0
        M4     4.0    5.0
        M5    10.0    NaN
        M6    10.0    NaN
        M7    10.0    NaN
        M8     9.0    NaN
        M9    10.0    NaN
        W1     NaN   10.0
        W2     NaN   10.0
        W3     1.0    8.0
...            ...    ...
09H     M7    10.0    NaN
        M8    10.0    NaN
        M9    10.0    NaN
        W1     NaN   10.0
        W2     NaN   10.0
        W3     NaN   10.0
        W4     NaN   10.0
        W5     NaN   10.0
        W6     8.0    2.0
        W7    10.0    NaN
        W8    10.0    NaN
        W9    10.0    NaN
10H     M1     NaN   10.0
        M2     NaN   10.0
        M3     NaN   10.0
        M4     NaN   10.0
        M5     2.0    8.0
        M6     8.0    2.0
        M7    10.0    NaN
        M8    10.0    NaN
        M9    10.0    NaN
        W1     NaN   10.0
        W2     NaN   10.0
        W3     NaN   10.0
        W4     NaN   10.0
        W5     NaN   10.0
        W6     3.0    7.0
        W7     9.0    NaN
        W8    10.0    NaN
        W9    10.0    NaN

[180 rows x 2 columns]

In [21]:
evaluations[evaluations.isnull()] = 0
evaluations.head()

Evaluation    gros  mince
Subject Item             
01F     M1     0.0    8.0
        M2     0.0    9.0
        M3     0.0    9.0
        M4     1.0    7.0
        M5    10.0    0.0

In [22]:
evaluations['prop'] = evaluations['gros']/(evaluations['mince']+evaluations['gros'])
evaluations.head(20)

Evaluation    gros  mince      prop
Subject Item                       
01F     M1     0.0    8.0  0.000000
        M2     0.0    9.0  0.000000
        M3     0.0    9.0  0.000000
        M4     1.0    7.0  0.125000
        M5    10.0    0.0  1.000000
        M6    10.0    0.0  1.000000
        M7    10.0    0.0  1.000000
        M8     9.0    0.0  1.000000
        M9    10.0    0.0  1.000000
        W1     0.0    7.0  0.000000
        W2     0.0   10.0  0.000000
        W3     0.0   10.0  0.000000
        W4     1.0    8.0  0.111111
        W5     8.0    0.0  1.000000
        W6    10.0    0.0  1.000000
        W7     8.0    0.0  1.000000
        W8    10.0    0.0  1.000000
        W9    10.0    0.0  1.000000
02F     M1     0.0   10.0  0.000000
        M2     0.0    9.0  0.000000

In [ ]:
#On case les index et on creer une colonne ItemSize

evaluations = evaluations.reset_index()

def create_ItemGender(text):
    return text[0]

def create_ItemSize(text):
    return float(text[1])

evaluations['ItemGender'] = evaluations['Item'].apply(create_ItemGender)
evaluations['ItemSize'] = evaluations['Item'].apply(create_ItemSize)
evaluations.head(25)

data['ItemGender'] = data['Item'].apply(create_ItemGender)
data['ItemSize'] = data['Item'].apply(create_ItemSize)

evaluations